In [1]:
import pandas as pd
import numpy as np
import torch
import datetime
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker 
import folium
import branca
from datetime import datetime, timedelta,date
from scipy.interpolate import make_interp_spline, BSpline
import plotly.express as px
import json, requests
import calmap

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 99)
from tqdm import tqdm
import plotly.express as px
import plotly.subplots as subplots
import plotly.graph_objects as go
import seaborn as sns
import os
import matplotlib.pyplot as plt
from PIL import Image

from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
def get_benchmark(model_dir, model_name, location2name, target='1 wk ahead cum death'):
    baseline_fps = os.listdir(os.path.join(model_dir, model_name))
    baseline_fps = [item for item in baseline_fps if not (item.startswith('metadata') or item.endswith('.txt'))]
    horizon = 7 * int(target.split()[0])
    try:
        if len(baseline_fps)==0:
            return -1
        for i,_fp in enumerate(baseline_fps):
            tmp = pd.read_csv(os.path.join(model_dir, model_name, _fp))
            tmp = tmp[tmp['type']=='point'][tmp['target']==target]
            tmp['region'] = tmp.location.map(location2name)
            tmp = tmp[tmp.region!='US'][['forecast_date','target_end_date','value','region']]
            tmp['target_start_date'] = pd.to_datetime(tmp['target_end_date']) - timedelta(days=horizon-1)
            tmp['target_end_date'] = pd.to_datetime(tmp['target_end_date'])
            if i==0:
                df = tmp.copy()
            else:
                df = pd.concat([df,tmp], axis=0)

        return df.sort_values(['target_start_date','region']).reset_index(drop=True)
    except:
        return -1

def get_label(death_fp, target_start_date, horizon=7, group_name='Province_State'):
    deaths = pd.read_csv(death_fp)
    ts_features = [item for item in deaths.columns if item not in ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
           'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population', 'Province/State', 'Country/Region', 'Long']]

    deaths_us = deaths.groupby(group_name)[ts_features].sum().diff(axis=1).T
    deaths[ts_features] = deaths[ts_features].mask(deaths[ts_features]<0,0)
    deaths_us.index = pd.to_datetime(deaths_us.index)
    use_index = pd.date_range(start=target_start_date,freq='D',periods=horizon)
    deaths_us = deaths_us[deaths_us.index.isin(use_index)]
    deaths_us['target_start_date'] = pd.to_datetime(use_index[0])
    deaths_us['target_end_date'] = pd.to_datetime(use_index[-1])
    deaths_us['horizon'] = horizon
    deaths_us = deaths_us.set_index(['target_start_date','target_end_date','horizon']).stack().reset_index().rename({0:'label', group_name:'region'},
                                                                                          axis=1)
    deaths_us = deaths_us.groupby(['target_start_date','horizon','region'])['label'].sum().reset_index()
    deaths_us.loc[deaths_us['label']<0,'label'] = 0

    deaths_cum =  deaths.groupby(group_name)[ts_features].sum().T
    deaths_cum.index = pd.to_datetime(deaths_cum.index)
    deaths_cum = deaths_cum.loc[pd.to_datetime(target_start_date)- datetime.timedelta(days=1)].reset_index()
    deaths_cum.columns = ['region','cum_label']
    deaths_us = pd.merge(deaths_us,deaths_cum,how='left',on=['region'])
    deaths_us['cum_label'] = deaths_us['cum_label'] + deaths_us['label']
    
    return deaths_us


In [3]:
model_dir = '/home/zhgao/covid19-forecast-hub/data-processed/'
location_fp = '/home/zhgao/covid19-forecast-hub/data-locations/locations.csv'
death_fp = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
confirmed_fp ='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
location = pd.read_csv(location_fp)
location2name = dict(zip(location['location'],location['location_name']))  

In [4]:
model_name_list = [item for item in os.listdir(model_dir) if '.' not in item]

In [5]:
model_results = dict()
for model_name in model_name_list:
    tmp = get_benchmark(model_dir,model_name,location2name)
    if isinstance(tmp,int) and tmp==-1:
        print("Empty results: {}".format(model_name))
    elif len(tmp)==0:
        print("Zero results: {}".format(model_name))
    else:
        model_results[model_name]=tmp

Zero results: Imperial-ensemble1
Zero results: CDDEP-SEIR_MCMC
Zero results: QJHong-Encounter
Empty results: ISUandPKU-vSEIdR
Zero results: Quantori-Multiagents
Zero results: PandemicCentral-USCounty
Empty results: Caltech-CS156
Zero results: Yu_Group-CLEP
Empty results: UCM_MESALab-FoGSEIR
Zero results: Imperial-ensemble2
Zero results: CMU-TimeSeries
Zero results: UMass-ExpertCrowd
Zero results: IBF-TimeSeries
Zero results: IQVIA_ACOE-STAN
Zero results: UVA-Ensemble
Zero results: UCF-AEM
Zero results: UChicagoCHATTOPADHYAY-UnIT


In [123]:
model_name = 'MSRA-DeepST'
tmp = get_benchmark('/home/zhgao/COVID-Research/','CDC',location2name)
#tmp = get_benchmark('/home/zhgao/covid19-forecast-hub/data-processed/',model_name,location2name)
model_results[model_name]=tmp

In [127]:
forecast_start_date = ['2020-09-21','2020-09-28','2020-10-05','2020-10-12','2020-10-19','2020-10-26','2020-11-02','2020-11-09','2020-11-16'][-1:]
epiweek_end_dates = [pd.to_datetime(start_date) + timedelta(days=5) for start_date in forecast_start_date]
epiweek_start_dates = [pd.to_datetime(start_date) - timedelta(days=1) for start_date in forecast_start_date]
deaths = pd.read_csv(death_fp)
ts_features = [item for item in deaths.columns if item not in ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population', 'Province/State', 'Country/Region', 'Long']]
deaths = deaths.groupby('Province_State')[ts_features].sum()
deaths = deaths.stack().reset_index().rename({'Province_State':'region','level_1':'target_end_date',0:'label'},axis=1)
deaths['target_end_date'] = pd.to_datetime(deaths['target_end_date'])
deaths = deaths[deaths['target_end_date'].isin(epiweek_end_dates)]

In [131]:
regions = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

score_results = dict()
score_results_all = dict()
score_tmp = dict()
for k,v in  model_results.items():
    v = v.drop_duplicates(['region','target_end_date'],keep='first')
    tmp = pd.merge(deaths,v,on=['region','target_end_date'],how='inner')
    if k in ['MSRA-DeepST','USC-SI_kJalpha']:
        print(k)
        print(tmp)
    dates = tmp['forecast_date'].unique()
    region_size = tmp['region'].unique().shape[0]
    score_tmp[k] = tmp
    tmp = tmp[tmp['region'].isin(regions)]
    if len(tmp['region'].unique()) > 30:
        score = (tmp['label'] - tmp['value']).map(abs).mean()
        score_results[k] = score
        
        score_all = tmp.groupby('target_end_date').apply(lambda x: (x['label'] - x['value']).map(abs).mean())
        score_results_all[k] = score_all
    else:
        print(k)

UChicago-CovidIL
MSRA-DeepST
            region target_end_date  label forecast_date         value  \
0          Alabama      2020-11-21   3457    2020-11-16   3402.413796   
1           Alaska      2020-11-21    102    2020-11-16    115.283641   
2          Arizona      2020-11-21   6457    2020-11-16   6480.979095   
3         Arkansas      2020-11-21   2337    2020-11-16   2284.455713   
4       California      2020-11-21  18672    2020-11-16  18643.045290   
5         Colorado      2020-11-21   2787    2020-11-16   2733.106411   
6      Connecticut      2020-11-21   4828    2020-11-16   4818.163515   
7         Delaware      2020-11-21    746    2020-11-16    758.662447   
8          Florida      2020-11-21  17930    2020-11-16  17921.106867   
9          Georgia      2020-11-21   9179    2020-11-16   9305.387748   
10          Hawaii      2020-11-21    231    2020-11-16    227.034938   
11           Idaho      2020-11-21    847    2020-11-16    837.945438   
12        Illinois    

UChicago-CovidIL_10_+
Google_Harvard-CPF
RPI_UW-Mob_Collision
GT_CHHS-COVID19
UChicago-CovidIL_30_+
Covid19Sim-Simulator
USC-SI_kJalpha_RF
IHME-CurveFit
NotreDame-FRED
MITCovAlliance-SIR
UChicago-CovidIL_80
Auquan-SEIR
UCSB-ACTS
UChicago-CovidIL_40
JHU_UNC_GAS-StatMechPool


In [126]:
#pd.DataFrame(score_results_all).T.sort_values('2020-11-21')


target_end_date,2020-09-26,2020-10-03,2020-10-10,2020-10-17,2020-10-24,2020-10-31,2020-11-07,2020-11-14,2020-11-21
Karlen-pypm,22.418000,19.626000,28.114000,30.154000,33.052000,36.080000,47.314000,35.144000,34.122000
UMass-MechBayes,25.580000,22.220000,26.640000,21.960000,32.860000,31.480000,47.080000,41.460000,36.840000
MSRA-DeepST,19.578698,19.954740,23.727839,17.395617,24.736893,17.218152,33.959902,26.751349,37.003657
OliverWyman-Navigator,21.725580,28.035754,27.592011,26.975128,33.928473,31.797894,41.686220,36.484265,37.516168
BPagano-RtDriven,NaN,NaN,28.040200,34.023660,38.829520,42.012520,54.710460,44.064460,37.530400
LNQ-ens1,22.554527,20.061792,25.131380,20.511517,34.568366,29.862089,41.541590,39.668976,38.522131
GT-DeepCOVID,28.627447,22.287184,25.689713,31.437827,42.332670,30.161069,43.124458,36.599502,40.949693
USC-SI_kJalpha,27.400000,29.040000,25.420000,27.200000,35.900000,37.540000,46.480000,37.600000,41.080000
LANL-GrowthRate,26.884523,24.640661,34.432593,30.232065,55.229018,36.663110,45.780000,72.320000,44.950000
SteveMcConnell-CovidComplete,21.640000,29.980000,28.960000,24.340000,33.700000,35.620000,47.880000,34.640000,45.020000


In [46]:
stats = pd.DataFrame(score_results_all).T.rank().sort_values('2020-10-31').fillna(99).astype(int)
stats.columns = ['Epiweek_{}'.format(str(week).split()[0]) for week in stats.columns]
stats[:10].style.background_gradient(cmap='viridis_r').set_precision(0)

KeyError: '2020-10-31'

In [210]:
dump_dict = dict()
tmp = stats[:10].reset_index()
for col in tmp.columns:
    dump_dict[col] = tmp[col].values.tolist()
SAVE_DIR = '../website/'
with open(os.path.join(SAVE_DIR, 'MSRA.Deaths.Predict.Rank.json'),'w') as f:
    f.write(json.dumps(dump_dict))

521

In [ ]:
stats = pd.DataFrame(score_results_all).T.sort_values('2020-10-31').fillna(99.0)[:10].T
maes = stats.mean()

In [ ]:
stats.columns = [item + '('+str(round(score,4))+')' for item,score in zip(stats.columns, maes.values)]
stats = stats.unstack().reset_index().rename({'level_0':'TeamName',
                                              'target_end_date':'Date',
                                              0:'MAE'},axis=1)

In [ ]:
fig = px.line(stats, x='Date', y='MAE',color='TeamName')
_ = fig.update_layout(title_text='MAE performance of top 10 teams')
fig.show()

In [25]:
regions = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington']

confirmed = pd.read_csv(confirmed_fp)
ts_features = [item for item in confirmed.columns if item not in ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Population', 'Province/State', 'Country/Region', 'Long']]
confirmed = confirmed.groupby('Province_State')[ts_features].sum()
confirmed = confirmed.stack().reset_index().rename({'Province_State':'region','level_1':'target_end_date',0:'label'},axis=1)
confirmed['target_end_date'] = pd.to_datetime(confirmed['target_end_date'])
#confirmed = confirmed[confirmed['target_end_date'].isin(epiweek_end_dates)]
inc_cases = confirmed.loc[confirmed['target_end_date'].isin(epiweek_end_dates), 'label'].values - confirmed.loc[confirmed['target_end_date'].isin(epiweek_start_dates), 'label'].values
confirmed = confirmed[confirmed['target_end_date'].isin(epiweek_end_dates)]
confirmed['label'] = inc_cases


confirmed_score_results = dict()
confirmed_score_tmp = dict()
for k,v in  model_results.items():
    #if k in ['COVIDhub-baseline','UMass-MechBayes']:
    #    continue
    v = v.drop_duplicates(['region','target_end_date'],keep='first')
    tmp = pd.merge(confirmed,v,on=['region','target_end_date'],how='inner')
    dates = tmp['forecast_date'].unique()
    region_size = tmp['region'].unique().shape[0]
    confirmed_score_tmp[k] = tmp
    tmp = tmp[tmp['region'].isin(regions)]
    if len(tmp['region'].unique()) > 30:
        score = (tmp['label'] - tmp['value']).map(abs).mean()
        confirmed_score_results[k] = score
    else:
        print(k)


UChicago-CovidIL
YYG-ParamSearch
SWC-TerminusCM
STH-3PU
UChicago-CovidIL_100
CovidActNow-SEIR_CAN
UChicago-CovidIL_60
UChicago-CovidIL_10_+
Google_Harvard-CPF
RPI_UW-Mob_Collision
GT_CHHS-COVID19
UChicago-CovidIL_30_+
IHME-CurveFit
NotreDame-FRED
MITCovAlliance-SIR
UChicago-CovidIL_80
USACE-ERDC_SEIR
Auquan-SEIR
UCSB-ACTS
UChicago-CovidIL_40
JHU_UNC_GAS-StatMechPool


In [175]:
pd.Series(confirmed_score_results).sort_values()

GT-DeepCOVID                    10219.673616
Covid19Sim-Simulator            10261.091418
CU-select                       10269.851064
CU-scenario_low                 10270.085106
BPagano-RtDriven                10274.711170
Karlen-pypm                     10275.172340
USC-SI_kJalpha                  10282.063830
CU-scenario_high                10282.170213
CU-nochange                     10282.276596
CU-scenario_mid                 10282.680851
JCB-PRM                         10283.486209
SteveMcConnell-CovidComplete    10284.404255
LANL-GrowthRate                 10286.680851
OliverWyman-Navigator           10287.533750
Geneva-DetGrowth                10288.408051
UMass-MechBayes                 10291.553191
MSRA-DeepST                     10291.667499
COVIDhub-ensemble               10292.702128
LNQ-ens1                        10292.733743
JHUAPL-Bucky                    10293.607667
UA-EpiCovDA                     10293.872340
UCSD_NEU-DeepGLEAM              10294.835454
MOBS-GLEAM

In [ ]:
results = pd.DataFrame({'Model':['MSRA-DeepST','LNQ-ens1','UMass-MechBayes','CDC-ensemble','JCB-PRM'],
                        '2020-10-11':[3,1,2,5,6],
                         '2020-10-04':[2,3,8,4,15],
                         '2020-09-27':[5,4,8,2,3],
                         '2020-09-20':[1,4,8,6,7],
                         '2020-10-18':[2,9,3,10,15],
                         '2020-10-25':[1,2,3,5,12]})